# EPA-based reverse engineering

In [ ]:
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters
from pyecsca.ec.mod import Mod, miller_rabin
from pyecsca.ec.point import Point
from pyecsca.ec.error import NonInvertibleError
from pyecsca.ec.mult import LTRMultiplier

In [ ]:
model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
add = coords.formulas["add-2007-bl"]
dbl = coords.formulas["dbl-2007-bl"]
neg = coords.formulas["neg"]

p = 0xdb49063db56b7783fa01dd62077c5a88dfa28009  # such that phi(p)/p ~ 0.5
a = Mod(0xaee572fdd4790bcd4729bb3b612b52a573df46e9, p)
b = Mod(0xdab9e68366a593ca1df9cb2f20890a578729d6ef, p)

n = 0x34a9fbe62b272f930b2e5027780a32300feb0dd8f
h = 1
gx = Mod(0xd4a3aaf43bdb25be7c308b69ae54f639e6e32e8c, p)
gy = Mod(0x7b6c82140bb427ac6e2a64507f60775949b2c8ce, p)

g = Point(coords, X=gx, Y=gy, Z=Mod(1, p))
infty = Point(coords, X=Mod(0, p), Y=Mod(1, p), Z=Mod(0, p))

curve = EllipticCurve(model, coords, p, infty, dict(a=a, b=b))
params = DomainParameters(curve, g, n, h)
print(miller_rabin(p))
print(curve.is_on_curve(g))

In [ ]:
mult = LTRMultiplier(add, dbl, None, False, True, True)
mult.init(params, g)
non = 0
for _ in range(1000):
    scalar = int(Mod.random(n*5))
    
    res = mult.multiply(scalar)
    #print(res)
    try:
        res.Z.inverse()
        print("yay", scalar, res)
        break
    except NonInvertibleError as e:
        non += 1
print(non)

In [ ]:
b.inverse()

In [ ]:
b

In [ ]:
dbl(p, g, a=a, b=b)

In [ ]:
Mod(753921130225168876277183530034689476795287082245, p).inverse()

In [ ]:
curve.affine_random()